# Train detector

## Mount Google Drive
To begin, let's mount our google drive using the `mount` function from `google.colab.drive`.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Clone/update repository


In order to get access to scripts used in this notebook we need to clone the required GitHub repository. This step is needed only the first time.

In [ ]:
%%bash

DIR="/content/gdrive/MyDrive/School_in_AI_PW/project_folder"
if [ -d "$DIR" ]; then
  rm -r -f "$DIR"
fi
mkdir $DIR
cd $DIR
git clone https://github.com/sir3mat/SchoolInAI_ProjectWork.git

## Update and install required dependencies

In [ ]:
%%bash
cd /content/gdrive/MyDrive/School_in_AI_PW/project_folder
pip install coloredlogs torchinfo
pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

## Inference detector

In [1]:
INPUT_IMAGE_PATH="/content/gdrive/MyDrive/School_in_AI_PW/storage/input_examples/0001.jpg"
MODEL_PATH="/content/gdrive/MyDrive/School_in_AI_PW/storage/motsynth_output/detection_logs/fasterrcnn_training/checkpoint.pth"
THRESHOLD=0.75

In [ ]:
import torch
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FastRCNNPredictor
from src.detection.graph_utils import show, add_bbox
from torchvision import transforms
from PIL import Image

checkpoint = torch.load(MODEL_PATH, map_location="cpu")

device = torch.device("cuda")
model = fasterrcnn_resnet50_fpn()
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2) 
model.load_state_dict(checkpoint["model"])
model.to(device)
model = model.eval()

image = Image.open(INPUT_IMAGE_PATH).convert('RGB')
transform = transforms.Compose([
transforms.ToTensor(),
])
image = transform(image).to(device)
image = image.unsqueeze(0)

with torch.no_grad():
        prediction = model(image)[0]

output = add_bbox(image[0], prediction, THRESHOLD)
show(output)